# KernelArgs

zum hochladen auf das device brauchen wir einen host pointer, der auf das Objekt zeigt und die größe des Objektes weiß

## typen

 - 2d dimensionale arrays lassen sich nicht trivial kopieren

 - fundamental types wie int, float müssen weder kopiert noch alloziert werden, es macht Sinn für diese overloads anzubieten.

 - alles andere muss alloziert werden
     - für diese objekte machen z.B. smart pointer sinn, da diese es zwar nicht unmöglich machen einen pointer auf einen pointer zu richten, aber schwer
     - andere möglichkeit wäre eventuell std::any, da dies type safe ist
     
______________

### funktions overload für fundamental types
funktionen nur für alle fundamental types zu templaten ist möglich und sollten sich leicht instantieren lassen

In [1]:
#include <iostream>
#include <string>

template<typename T, typename = typename std::enable_if<std::is_fundamental<T>::value && std::is_const<T>::value>::type>
std::string KernelArg(T & test) {
    return "const fundamental";
}

template<typename T, typename = typename std::enable_if<!std::is_const<T>::value && std::is_fundamental<T>::value>::type>
std::string KernelArg(T *test) {
    return "mutable fundamental";
}


In [2]:
int mut_int{5};
const int const_int{5};

In [3]:
KernelArg(&mut_int)

"mutable fundamental"

In [4]:
KernelArg(const_int)

"const fundamental"

### KernelArg: smart pointer
Hier mein erster Ansatz, der entweder shared_ptr oder fundamental types nimmt,
wobei diese in *const referenzen* und *mutable pointer* unterschieden werden

const referenzen werden nicht wieder heruntergeladen

In [5]:
#include <memory>

class KernelArg1 {
    
    public:
        KernelArg1(std::shared_ptr<void> hdata, size_t size, bool download = false, bool copy = true, bool upload = true) : 
            m_hdata{hdata}, m_size{size}, m_download{download}, m_copy{copy}, m_upload{upload} {}
    
       
        template<typename T, typename = typename std::enable_if<!std::is_const<T>::value && std::is_fundamental<T>::value>::type>
        KernelArg1(T *hdata, bool download) : 
            m_hptr{hdata}, m_size{0}, m_download{download}, m_copy{false}, m_upload{false} {}
    
        template<typename T, typename = typename std::enable_if<std::is_fundamental<T>::value && std::is_const<T>::value>::type>
        KernelArg1(T & hdata) :
            m_hptr{&hdata}, m_size{0}, m_download{false}, m_copy{false}, m_upload{false} {}
    
//         KernelArg1(int *hdata, bool download) : 
//             m_hptr{hdata}, m_size{0}, m_download{download}, m_copy{false}, m_upload{false} {}
    
//         KernelArg1(const int & hdata) :
//             m_hptr{&hdata}, m_size{0}, m_download{false}, m_copy{false}, m_upload{false} {}
    
        std::string upload() {
            if (m_hptr) {
                return "uploading m_hptr " + std::to_string(reinterpret_cast<intptr_t>(const_cast<void*>(m_hptr)));
            }
            return "uploading m_hdata " + std::to_string(reinterpret_cast<intptr_t>(m_hdata.get()));
        }

    private:
        std::shared_ptr<void> m_hdata; // host data
        const void* m_hptr = nullptr;  // host data
        const size_t m_size;   // size of object
        const bool m_copy;     // memcopyHtoD
        const bool m_download; // memcopyDtoH
        const bool m_upload;   // allocate on host
}

anstatt m_hptr und m_hdata könnte man das vielleicht auch mit einem Union oder std::variant zusammenfassen

### Benutzung von KernelArg1

In [6]:
typedef struct {
    short red;
    short green;
    short blue;
} pixel;

nun können wir ein pixel array erstellen und wie gewohnt benutzen

In [7]:
const size_t SIZE = 64;
std::shared_ptr<pixel[]> image(new pixel[SIZE]);

for (size_t i{0}; i < SIZE; ++i) {
        image[i].red = i;
        image[i].blue = i*2;
        image[i].green = i*3;
    }
    
std::cout << "image[32].red:          " << image[32].red << std::endl;
std::cout << "image[32].blue:         " << image[32].blue << std::endl;
std::cout << "image[32].green:        " << image[32].green << std::endl;

image[32].red:          32
image[32].blue:         64
image[32].green:        96


nun wollen wir hiermit KernelArgs erstellen und diese hochladen,
falsche größen könnten hier nachher beim downloaden zu segfaults führen

In [8]:
std::vector<KernelArg1> args1;
args1.emplace_back(KernelArg1{image, SIZE * sizeof(float)}); // input array
args1.emplace_back(KernelArg1{SIZE}); // constante

for (auto &arg : args1) {
    std::cout << arg.upload() << std::endl;
}

uploading m_hdata 94121619169680
uploading m_hptr 140349579019848


#### std::vector
mit vektoren zu arbeiten, könnte sich hier anbieten, 
da diese ihre größe immer mitgeben

std::array zwar auch, da dieses aber nur auf dem stack existiert, kommt es bei matrizen > 500 schnell auch zu segfaults

In [9]:
#include <vector>

// nicht nützlich in einer library
template <typename T>
size_t KernelArgVektor(std::vector<T> v) {
    return v.size() * sizeof(T);
}

// nur fundamental types, möglich aber stark eingeschränkt
template<typename T, 
    typename = typename std::enable_if<std::is_fundamental<T>::value>::type>
size_t KernelArgVektorFundamental(std::vector<T> v) {
    return v.size() * sizeof(T);
}

In [10]:
std::vector<unsigned int> v = {7, 5, 16, 8};
v.resize(10);

In [11]:
KernelArgVektor(v)

40

In [12]:
KernelArgVektorFundamental(v)

40

dies muss aber entweder getemplated werden oder kann nur für fundamental Types gemacht werden, 
__keine__ benutzer definierten Typen, wie der Pixel Strukt

In [13]:
std::vector<pixel> image_vector;
image_vector.resize(SIZE);
int i{0};
for (auto &image_pixel : image_vector) {
    image_pixel.red = i*3;
    image_pixel.blue = i*2;
    image_pixel.green = i++;
}

std::cout << "image_vector[32].red:   " << image_vector[32].red << std::endl;
std::cout << "image_vector[32].blue:  " << image_vector[32].blue << std::endl;
std::cout << "image_vector[32].green: " << image_vector[32].green << std::endl;

image_vector[32].red:   96
image_vector[32].blue:  64
image_vector[32].green: 32


ich weiß auch nicht, ob ein shared_ptr zu einem Vektor nicht eine *schlechte Idee* ist, 
da vektoren selbst schon ein array verwalten

wird dann wenn der shared_ptr den scope verlässt der destruktor von dem vektor 
und dem smart pointer auf den selben pointer aufgerufen?

In [14]:
std::shared_ptr<pixel> image_vector_ptr(image_vector.data());

In [15]:
std::vector<KernelArg1> args2;
args2.emplace_back(KernelArg1{image, SIZE * sizeof(float)});
args2.emplace_back(KernelArg1{SIZE});

for (auto &arg : args2) {
    std::cout << arg.upload() << std::endl;
}

uploading m_hdata 94121619169680
uploading m_hptr 140349579019848


### KernelArg2: move semantic
eine Überlegung wäre auch ob man nicht Ownership der KernelArg Klasse übergeben sollte, 
damit man zwischen dem erstellen von dem KernelArg und dem hochladen nicht die Objekte dealloziert oder 
z.B. einen Vektor resized

Also:

  1. auto obj = make_unique
  2. auto device_obj = KernelArg(std::move(obj))
  3. kernel.launch()
  4. auto host_obj = device_obj.download()

In [16]:
class KernelArg2 {
    
    public:
        KernelArg2(std::shared_ptr<void> hdata, size_t size, bool download = false, bool copy = true, bool upload = true) : 
            m_hdata{hdata}, m_size{size}, m_download{download}, m_copy{copy}, m_upload{upload} {}
    
        //fundamental types, hier nur int
        KernelArg2(int *hdata, bool download) : 
            m_hptr{hdata}, m_size{0}, m_download{download}, m_copy{false}, m_upload{false} {}
        KernelArg2(const int &hdata) :
            m_hptr{&hdata}, m_size{0}, m_download{false}, m_copy{false}, m_upload{false} {}
    
        std::shared_ptr<void> download() {

            return std::move(m_hdata);
        }
    
    private:
        std::shared_ptr<void> m_hdata; // host data
        const void* m_hptr = nullptr;  // host data
        const size_t m_size;   // size of object
        const bool m_copy;     // memcopyHtoD
        const bool m_download; // memcopyDtoH
        const bool m_upload;   // allocate on host
}


In [17]:
std::shared_ptr<pixel[]> image2(new pixel[SIZE]);

for (size_t i{0}; i < SIZE; ++i) {
    image2[i].red = i;
    image2[i].blue = i*2;
    image2[i].green = i*3;
}

std::vector<KernelArg2> args3;

auto device_array = KernelArg2{std::move(image2), SIZE * sizeof(float), true};

args3.push_back(device_array); // input array
args3.emplace_back(KernelArg2{SIZE}); // constante

// std::cout << image2[7].red << std::endl; // leads to UB, move macht es eher schlechter

// launch kernel

auto output_array = std::static_pointer_cast<pixel[]>(device_array.download());

std::cout << "image2[32].red:   " << output_array[32].red << std::endl;
std::cout << "image2[32].blue:  " << output_array[32].blue << std::endl;
std::cout << "image2[32].green: " << output_array[32].green << std::endl;

image2[32].red:   32
image2[32].blue:  64
image2[32].green: 96


In [18]:
std::vector<int> test_vector = {1,2,3,4,5,6,7,8};
test_vector.resize(64);
std::shared_ptr<int[]> test_vector_ptr(test_vector.data());

auto test_array1 = KernelArg2{test_vector_ptr, test_vector.size() * sizeof(int), true};
test_vector.resize(32); // should throw compiler error, hard to detect?
test_vector_ptr[7]

8

## Zusammenfassend

- move semantik hilft nicht wirklich viel und verlängert nur host code
- shared_ptr machen es schwieriger segfaults zu erreichen, aber nicht unmöglich
- size des allozierten bereichs muss manuell berechnet werden => potentielle segfaults

____________________________________________

In [19]:
struct Foo {
    Foo(int n = 0) noexcept : bar(n) {
        std::cout << "Foo: constructor, bar = " << bar << '\n';
    }
    ~Foo() {
         std::cout << "Foo: destructor, bar = " << bar << '\n';
    }
    int getBar() const noexcept { return bar; }
private:
    int bar;
};

In [20]:
auto sptr = std::make_shared<Foo>(1);
std::cout << "The first Foo's bar is " << sptr->getBar() << "\n";

auto foo_ptr = KernelArg2{std::move(sptr), sizeof(Foo)};

sptr.reset(new Foo);

std::cout << "The second Foo's bar is " << sptr->getBar() << "\n";

// launch kernel

Foo: constructor, bar = 1
The first Foo's bar is 1
Foo: constructor, bar = 0
The second Foo's bar is 0


@0x7fa5b599bb40

### KernelArg2: unique_ptr 
unique_ptr würden move semantic erzwingen und klar machen, 
dass bis zum downloaden der ptr dem Kernel gehört
leider ist das wesentlich komplizierter, da unique_ptr keine *type-erasure* implementiert

In [21]:
// class KernelArg3 {
    
//     public:
//         KernelArg3(std::unique_ptr<void> hdata, size_t size, bool download = false, bool copy = true, bool upload = true) : 
//             m_hdata{std::move(hdata)}, m_size{size}, m_download{download}, m_copy{copy}, m_upload{upload} {}
    
//         //fundamental types, hier nur int
//         KernelArg3(int *hdata, bool download) : 
//             m_hptr{hdata}, m_size{0}, m_download{download}, m_copy{false}, m_upload{false} {}
//         KernelArg3(const int &hdata) :
//             m_hptr{&hdata}, m_size{0}, m_download{false}, m_copy{false}, m_upload{false} {}
    
//         std::unique_ptr<void> download() {
//             // return type std::variant<std::unique_ptr<void>, void*>
//             // falls m_hptr ??
//             return std::move(m_hdata);
//         }
    
//     private:
//         std::unique_ptr<void> m_hdata; // host data
//         const void* m_hptr = nullptr;  // host data
//         const size_t m_size;   // size of object
//         const bool m_copy;     // memcopyHtoD
//         const bool m_download; // memcopyDtoH
//         const bool m_upload;   // allocate on host
// }

In [22]:
// auto image2 = std::make_unique<pixel[]>(SIZE);

// for (size_t i{0}; i < SIZE; ++i) {
//     image2[i].red = i;
//     image2[i].blue = i*2;
//     image2[i].green = i*3;
// }

In [23]:
// std::vector<KernelArg2> args3;

// auto device_array = KernelArg2{std::move(image2), SIZE * sizeof(float), true};

// image2[5].red = 5; 

// args3.push_back(device_array); // input array
// args3.emplace_back(KernelArg2{SIZE}); // constante

// // launch kernel

// auto output_array = device_array.download();

// std::cout << "image2[32].red:          " << output_array[32].red << std::endl;
// std::cout << "image2[32].blue:         " << output_array[32].blue << std::endl;
// std::cout << "image2[32].green:        " << output_array[32].green << std::endl;